* Nearly 40seconds for processing single image on A100 machine
* Generates the same words again and again
* Processing Time Increases as the prompt size increases
* small prompts are working fine 
* for small prompts taking 4 to 5 seconds for an image

In [2]:
%pip install -U transformers
%pip install accelerate
%pip install huggingface_hub
%pip install pillow 
%pip install pdf2image


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
from pdf2image import convert_from_path

images = convert_from_path("/home/Abraham Rodriguez OCR_removed.pdf")

In [4]:
import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForVision2Seq
from transformers.image_utils import load_image
from datetime import datetime

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"


# Initialize processor and model
# processor = AutoProcessor.from_pretrained("HuggingFaceTB/SmolVLM-Base")
# model = AutoModelForVision2Seq.from_pretrained(
#     "HuggingFaceTB/SmolVLM-Base",
#     torch_dtype=torch.bfloat16,
#     # _attn_implementation="flash_attention_2" if DEVICE == "cuda" else "eager",
# ).to(DEVICE)

processor = AutoProcessor.from_pretrained("HuggingFaceTB/SmolVLM-Instruct")
model = AutoModelForVision2Seq.from_pretrained("HuggingFaceTB/SmolVLM-Instruct").to(DEVICE)





/root/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some kwargs in processor config are unused and will not have any effect: image_seq_len. 


In [12]:
from datetime import datetime
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image"},
            {"type": "text", "text": """You are a helpful assistant. Please extract all the text from the image exactly as it appears, including formatting, line breaks, and special characters, and provide it to me in markdown format."""}
        ]
    },
]
# Prepare inputs


for idx,image in enumerate(images):
    print(f"Page No - {idx+1} start-time - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    prompt = processor.apply_chat_template(messages, add_generation_prompt=True)
    inputs = processor(text=prompt, images=[image], return_tensors="pt")
    inputs = inputs.to(DEVICE)
    generated_ids = model.generate(**inputs, max_new_tokens=2000)
    generated_texts = processor.batch_decode(
        generated_ids,
        skip_special_tokens=True,
    )
    print(f"Page No - {idx+1} end-time - {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    with open(f"/home/page_{idx+1}.md", "w", encoding="utf-8") as md_file:
        md_file.write(generated_texts[0])
    print(generated_texts)
    


Page No - 1 start-time - 2025-01-30 03:41:53
Page No - 1 end-time - 2025-01-30 03:42:04
['User:<image>You are a helpful assistant. Please extract all the text from the image exactly as it appears, including formatting, line breaks, and special characters, and provide it to me in markdown format.\nAssistant: ### AFFIDAVIT OF MEDICAL RECORDS\n\n#### (Pursuant to Texas Rules of Evidence 902(10)(b))\n\n### Medical Records Pertaining to:\n\n### Abraham Rodriguez\n\n### Before me, the undersigned authority, personally appeared\n\n### David Rodriguez, who, being by me\n\n### affidavit, and personally acquainted with the facts herein stated.\n\n### My name is\n\n### ________________________________________\n\n### I am of sound mind, capable of making this\n\n### affidavit, and personally acquainted with the facts herein stated.\n\n### I am the custodian of the records of:\n\n### AHC Physicians\n\n### I am an employee or owner of AHC Physicians and am\n\n### familiar with the manner in which it